<a href="https://colab.research.google.com/github/monu322/LDA_Classification/blob/main/HDP_topics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyLDAvis

     |████████████████████████████████| 1.7MB 11.2MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 9.9MB 46.0MB/s 
     |████████████████████████████████| 15.3MB 269kB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-cp37-none-any.whl size=136870 sha256=aaa03920a897be7ac8cdd0864c1acf2d6b5fc9a1bf8098b37bccd59a297fe6f7
  Stored in directory: /root/.cache/pip/wheels/a0/9c/fc/c6e00689d35c82cf96a8adc70edfe7ba7904374fdac3240ac2
Successfully built pyLDAvis
ERROR: tensorflow 2.4.1 has requirement numpy~=1.19.2, but you'll have numpy 1.20.2 which is incompatible.
ERROR: google-colab 1.0.0 has requirement pandas~=1.1.0; python_version >= "3.0", but you'll have pandas 1.2.4 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 

In [ ]:
import pickle
import gensim
import pyLDAvis
import pyLDAvis.gensim_models
import spacy
import pandas as pd
import nltk; nltk.download('stopwords')
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import re
import warnings
from pprint import pprint

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_least_angle.py:169: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
df = pd.read_csv('yelp.csv')
df['text_len'] = df['text'].apply(lambda x: len(x.split()))

df.head()

,business_id,date,review_id,stars,text,type,user_id,cool,useful,funny,text_len
0,9yKzy9PApeiPPOUJEtnvkg,2011-01-26,fWKvX83p0-ka4JS3dc6E5A,5,My wife took me here on my birthday for breakf...,review,rLtl8ZkDX5vH5nAx9C3q5Q,2,5,0,155
1,ZRJwVLyzEJq1VAihDhYiow,2011-07-27,IjZ33sJrzXqU-0X6U8NwyA,5,I have no idea why some people give bad review...,review,0a2KyEL0d3Yb1V6aivbIuQ,0,0,0,257
2,6oRAC4uyJCsJl1X0WZpVSA,2012-06-14,IESLBzqUCLdSzSqm0eCSxQ,4,love the gyro plate. Rice is so good and I als...,review,0hT2KtfLiobPvh6cDC8JQg,0,1,0,16
3,_1QQZuf4zZOyFCvXc0o6Vg,2010-05-27,G-WvGaISbqqaMHlNnByodA,5,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",review,uZetl9T0NcROGOyFfughhg,1,2,0,76
4,6ozycU1RpktNG2-1BroVtw,2012-01-05,1uJFq2r5QfJG_6ExMRCaGw,5,General Manager Scott Petello is a good egg!!!...,review,vYmM4KTsC8ZfQBg-j5MWkw,0,0,0,86


In [ ]:
rev_train = df[df.stars != 3.0]
print('Number of reviews: ', len(rev_train))

Number of reviews:  8539


In [ ]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

def strip_newline(series):
    return [review.replace('\n','') for review in series]

rev_train['text'] = strip_newline(rev_train.text)
rev_train.text[21:22].values

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


array(["I love this place! I have been coming here for ages.My favorites: Elsa's Chicken sandwich, any of their burgers, dragon chicken wings, china's little chicken sandwich, and the hot pepper chicken sandwich. The atmosphere is always fun and the art they display is very abstract but totally cool!"],
      dtype=object)

In [ ]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

words_tr = list(sent_to_words(rev_train.text))

words_tr[21][:10]



['love',
 'this',
 'place',
 'have',
 'been',
 'coming',
 'here',
 'for',
 'ages',
 'my']

In [ ]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

words_tr = remove_stopwords(words_tr)

def bigrams(words, bi_min=15, tri_min=10):
    bigram = gensim.models.Phrases(words, min_count = bi_min)
    trigram = gensim.models.Phrases(bigram[words], min_count = tri_min)
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    trigram_mod = gensim.models.phrases.Phraser(trigram)
    return bigram_mod, trigram_mod

bigram_tr, trigram_tr = bigrams(words_tr)



/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [ ]:
print(trigram_tr[bigram_tr[words_tr[0]]][:200])

['wife', 'took', 'birthday', 'breakfast', 'excellent', 'weather', 'perfect', 'made', 'sitting_outside', 'overlooking', 'grounds', 'absolute', 'pleasure', 'waitress', 'excellent', 'food', 'arrived', 'quickly', 'semi', 'busy', 'saturday_morning', 'looked_like', 'place', 'fills', 'pretty_quickly', 'earlier', 'get', 'better', 'favor', 'get', 'bloody_mary', 'phenomenal', 'simply', 'best_ever', 'pretty_sure', 'use', 'ingredients', 'garden', 'blend', 'fresh', 'order', 'amazing', 'everything_menu', 'looks', 'excellent', 'white', 'truffle', 'scrambled_eggs', 'vegetable', 'skillet', 'tasty', 'delicious', 'came', 'pieces', 'griddled', 'bread', 'amazing', 'absolutely', 'made', 'meal', 'complete', 'best', 'toast', 'ever', 'anyway', 'wait_go_back']


In [ ]:
nlp = spacy.load('en', disable=['parser', 'ner'])

warnings.filterwarnings("ignore")

def lemmatization(texts, allowed_postags=['NOUN','ADJ','VERB','ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

trigrams_tr = [trigram_tr[bigram_tr[review]] for review in words_tr]

lemma_lg = lemmatization(trigrams_tr)

/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/usr/local/lib/python3.7/dist-packages/catalogue.py:126: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):


In [ ]:
lemma_lg[0][:20]

id2word_lg = gensim.corpora.Dictionary(lemma_lg)
id2word_lg.filter_extremes(no_below=10, no_above=0.35)
id2word_lg.compactify()
id2word_lg.save('train_dict_lg')
corpus_lg = [id2word_lg.doc2bow(text) for text in lemma_lg]

corpus_lg[22][:10]



[(9, 1),
 (13, 1),
 (14, 1),
 (20, 2),
 (55, 1),
 (59, 1),
 (63, 1),
 (70, 1),
 (86, 1),
 (88, 1)]

In [ ]:
[(id2word_lg[id], freq) for id, freq in corpus_lg[0]]

[('absolute', 1),
 ('absolutely', 1),
 ('amaze', 1),
 ('amazing', 1),
 ('arrive', 1),
 ('birthday', 1),
 ('blend', 1),
 ('bread', 1),
 ('breakfast', 1),
 ('busy', 1),
 ('come', 1),
 ('complete', 1),
 ('earlier', 1),
 ('ever', 1),
 ('excellent', 3),
 ('favor', 1),
 ('fill', 1),
 ('food', 1),
 ('fresh', 1),
 ('garden', 1),
 ('get', 1),
 ('ground', 1),
 ('ingredient', 1),
 ('look', 1),
 ('make', 2),
 ('meal', 1),
 ('order', 1),
 ('perfect', 1),
 ('phenomenal', 1),
 ('piece', 1),
 ('pleasure', 1),
 ('quickly', 1),
 ('semi', 1),
 ('simply', 1),
 ('skillet', 1),
 ('take', 1),
 ('tasty', 1),
 ('toast', 1),
 ('use', 1),
 ('vegetable', 1),
 ('waitress', 1),
 ('weather', 1),
 ('well', 1),
 ('white', 1),
 ('wife', 1)]

In [ ]:
from gensim.models import HdpModel
hdp = HdpModel(corpus_lg, id2word_lg, chunksize=10000)

len(hdp.print_topics())

20

In [ ]:
hdp.print_topics(num_topics=20)

[(0,
  '0.003*cream + 0.002*item + 0.002*anti + 0.002*absolutely + 0.002*critical + 0.002*cabbage + 0.002*trendy + 0.002*english + 0.002*sitting + 0.002*thought_would'),
 (1,
  '0.002*address + 0.002*tattoo + 0.002*utensil + 0.002*towner + 0.002*suit + 0.002*fortunately + 0.002*tooth + 0.002*sexy + 0.002*cherry + 0.002*marshmallow'),
 (2,
  '0.002*fair + 0.002*increase + 0.002*choose + 0.002*chewy + 0.002*spicy + 0.002*decline + 0.002*minimal + 0.002*soda + 0.002*taco + 0.002*meaning'),
 (3,
  '0.003*decadent + 0.002*topic + 0.002*speaker + 0.002*darn + 0.002*considering + 0.002*colorful + 0.002*service + 0.002*land + 0.002*cardboard + 0.002*couch'),
 (4,
  '0.003*adequate + 0.002*san + 0.002*desire + 0.002*upstairs + 0.002*tail + 0.002*goodness + 0.002*dipping_sauce + 0.002*chase + 0.002*veggie + 0.002*moment'),
 (5,
  '0.003*desire + 0.002*shopping + 0.002*dust + 0.002*pretzel + 0.002*trash + 0.002*condition + 0.002*stand + 0.002*spinach + 0.002*print + 0.002*blood'),
 (6,
  '0.002*e